## Înlănțuire (_chaining_)
Transformările și acțiunile se pot înlănțui (**chain**), obținându-se astfel secvențe/fluxuri de calcul (computation **pipelines**)
Presupunem ca vrem sa calculăm suma pătratelor 
$$ \sum_{i=1}^n x_i^2 $$
unde elementele $x_i$ sunt stocate intr-un RDD.

### Pornirea unui `SparkContext`

In [1]:
#Porniti un context Spark cu 4 core-uri


In [2]:
B=sc.parallelize(np.random.randint(0,10,size=1000))
lst = B.collect()
for i in lst: 
    print(i,end=', ')

8, 6, 0, 1, 1, 4, 1, 7, 0, 7, 4, 5, 4, 1, 2, 0, 5, 8, 5, 3, 7, 5, 4, 7, 5, 5, 5, 5, 5, 9, 3, 3, 3, 8, 9, 2, 2, 3, 7, 0, 6, 8, 5, 4, 8, 9, 3, 0, 5, 0, 1, 5, 8, 6, 6, 8, 4, 3, 9, 0, 5, 6, 9, 4, 6, 9, 5, 8, 1, 7, 6, 3, 7, 6, 7, 1, 8, 1, 7, 2, 1, 3, 6, 8, 9, 9, 9, 9, 6, 9, 4, 8, 5, 6, 8, 8, 7, 5, 8, 0, 4, 7, 1, 8, 6, 2, 8, 0, 3, 9, 5, 2, 7, 6, 2, 0, 1, 5, 6, 8, 8, 8, 2, 2, 2, 0, 3, 2, 1, 0, 0, 6, 7, 1, 4, 3, 2, 9, 4, 6, 1, 2, 1, 3, 7, 1, 4, 3, 9, 9, 5, 2, 5, 4, 1, 3, 6, 0, 2, 5, 0, 1, 0, 1, 6, 3, 8, 6, 8, 4, 6, 1, 2, 3, 3, 0, 4, 5, 6, 9, 3, 1, 0, 7, 3, 0, 0, 3, 0, 1, 0, 5, 2, 1, 5, 8, 6, 7, 8, 2, 8, 0, 3, 3, 7, 7, 1, 7, 0, 3, 3, 7, 7, 9, 4, 1, 8, 7, 6, 5, 9, 2, 5, 0, 3, 0, 3, 5, 1, 1, 1, 3, 8, 5, 2, 8, 1, 7, 4, 2, 5, 6, 7, 4, 5, 7, 3, 8, 7, 6, 1, 4, 7, 3, 2, 7, 1, 0, 9, 7, 9, 9, 2, 8, 0, 4, 4, 0, 2, 9, 8, 8, 6, 3, 2, 1, 1, 8, 4, 4, 6, 5, 5, 1, 6, 5, 0, 8, 7, 5, 5, 3, 2, 7, 0, 3, 7, 2, 1, 6, 6, 3, 5, 0, 0, 7, 5, 4, 6, 2, 8, 8, 8, 0, 7, 7, 6, 4, 9, 1, 2, 8, 5, 4, 6, 3, 6, 0, 8, 0, 3, 3, 6, 1

### Sintaxa secvențială pentru înlănțuire
Efectuăm o operație de atribuire după fiecare calcul

In [3]:
%%time
Squares=B.map(lambda x:x*x)
summation = Squares.reduce(lambda x,y:x+y)

Wall time: 2.48 s


In [4]:
print(summation)

28647


### Sintaxa în cascadă pentru înlănțuire
Combinarea calculelor într-o singura comandă în cascadă

In [5]:
%%time
B.map(lambda x:x*x).reduce(lambda x,y:x+y)

Wall time: 2.38 s


28647

### Ambele sintaxe reprezintă același lucru
Singurele diferente sunt următoarele:
* În sintaxa secvențială RDD-ul intermediar este denumit (`Squares`)
* În sintaxa în cascadă RDD-ul intermediar este *anonim*

Execuția este identică.

### Execuția secvențială
Modalitatea standard prin care sunt executate operațiile _map_ și _reduce_ este:
* se efectuează operația _map_
* RDD-ul rezultat este stocat în memorie
* se efectuează operația _reduce_

### Dezavantajele execuției secvențiale

1. Rezultatul intermediar (`Squares`) necesită spațiu de memorie.
2. Două parcurgeri ale memoriei (pentru `B`, apoi pentru `Squares`) - dublează numărul de eșecuri de regăsire a informației în _cache_ (_cache-misses_).

### Execuția de tip _pipeline_
Întregul calcul este efectuat printr-o singură parcurgere. Pentru fiecare element al lui **`B`**:
1. Se calculează pătratul elementului
2. Se transmite această valoare ca input al operației `reduce`.

### Avantajele execuției înlănțuite (_pipelined_)

1. Este necesară mai puțină memorie deoarece rezultatul intermediar nu este stocat.
2. Este mai rapida deoarece are loc o singură parcurgere a RDD-ului de intrare.

In [6]:
sc.stop()